# OpenAI Function Calling (Tools) Example

This notebook demonstrates how to use function calling with OpenAI in Dinnovos Agent.

In [ ]:
import os
import json
from dinnovos.llms.openai import OpenAILLM

## 1. Configure API Key

In [ ]:
# Configure your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY") or "your-api-key-here"
llm = OpenAILLM(api_key=api_key, model="gpt-4")

## 2. Define Tools (Functions)

In [ ]:
# Define available tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Bogotá, Colombia"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g. '2 + 2'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

print("Tools defined:", json.dumps(tools, indent=2))

## 3. Implement the Functions

In [ ]:
def get_weather(location: str, unit: str = "celsius") -> str:
    """Mock weather function"""
    return json.dumps({
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Sunny",
        "humidity": 65
    })

def calculate(expression: str) -> str:
    """Calculator function"""
    try:
        result = eval(expression)
        return json.dumps({"expression": expression, "result": result})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Dictionary of available functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

---

# 🚀 RECOMMENDED METHOD: call_with_function_execution()

**The easiest way** to use function calling. Automatically handles the entire cycle.

## Example 1: Basic Usage (Automatic)

In [ ]:
# One call does it all!
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the weather in Bogotá?"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True  # See the complete process
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Functions called: {len(result['function_calls'])}")

## Example 2: Multiple Functions

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Compare the weather in Bogotá and Medellín, and calculate 25 * 4"}],
    tools=tools,
    available_functions=available_functions,
    verbose=True
)

print(f"\n{'='*70}")
print("📝 FINAL RESPONSE:")
print(result['content'])
print(f"\n🔄 Iterations: {result['iterations']}")
print(f"🔧 Total functions called: {len(result['function_calls'])}")

## Example 3: Silent Mode

In [ ]:
# Without verbose - just get the result
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "Calculate (100 + 50) / 3"}],
    tools=tools,
    available_functions=available_functions,
    verbose=False
)

print("Response:", result['content'])

## Example 4: View Complete History

In [ ]:
result = llm.call_with_function_execution(
    messages=[{"role": "user", "content": "What's the temperature in Cali?"}],
    tools=tools,
    available_functions=available_functions
)

print("\n📊 COMPLETE ANALYSIS:")
print(f"\n✅ Response: {result['content']}")
print(f"\n🔄 Iterations performed: {result['iterations']}")
print(f"\n🔧 Functions executed:")
for i, func_call in enumerate(result['function_calls'], 1):
    print(f"\n  {i}. {func_call['name']}")
    print(f"     Args: {func_call['arguments']}")
    print(f"     Result: {func_call['result']}")

---

# Manual Methods (Advanced)

If you need full control over the process, you can use the low-level methods.

## 4. Example: Simple Call with Tools (Manual)

In [ ]:
messages = [
    {"role": "user", "content": "What's the weather in Bogotá?"}
]

response = llm.call_with_tools(messages, tools)
print("Initial response:")
print(json.dumps(response, indent=2))

In [ ]:
# Execute tool calls if they exist
if response["tool_calls"]:
    for tool_call in response["tool_calls"]:
        function_name = tool_call["function"]["name"]
        function_args = json.loads(tool_call["function"]["arguments"])
        
        print(f"\n🔧 Calling function: {function_name}")
        print(f"📋 Arguments: {function_args}")
        
        # Execute the function
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"✅ Function response: {function_response}")
        
        # Add the function response to messages
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": response["tool_calls"]
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call["id"],
            "name": function_name,
            "content": function_response
        })
    
    # Get final response
    final_response = llm.call_with_tools(messages, tools)
    print(f"\n💬 Final LLM response: {final_response['content']}")

## 5. Example: Streaming with Tools (Manual)

In [ ]:
messages = [
    {"role": "user", "content": "Calculate 123 * 456 + 789"}
]

print("Streaming response:")
tool_calls_buffer = {}

for chunk in llm.stream_with_tools(messages, tools):
    if chunk["type"] == "content":
        print(chunk["delta"], end="", flush=True)
    elif chunk["type"] == "tool_call":
        idx = chunk["index"]
        if idx not in tool_calls_buffer:
            tool_calls_buffer[idx] = {
                "id": "",
                "name": "",
                "arguments": ""
            }
        
        if chunk["tool_call_id"]:
            tool_calls_buffer[idx]["id"] = chunk["tool_call_id"]
        if chunk["function_name"]:
            tool_calls_buffer[idx]["name"] = chunk["function_name"]
        if chunk["function_arguments"]:
            tool_calls_buffer[idx]["arguments"] += chunk["function_arguments"]

print("\n")

In [ ]:
# Execute tool calls from streaming
if tool_calls_buffer:
    for idx, tool_call_data in tool_calls_buffer.items():
        function_name = tool_call_data["name"]
        function_args = json.loads(tool_call_data["arguments"])
        
        print(f"🔧 Calling function: {function_name}")
        print(f"📋 Arguments: {function_args}")
        
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"✅ Response: {function_response}")